In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from config import db_password,g_key
from config import resolution,lat_max,lat_min,lng_max,lng_min
from config import t_sat,flow_rate,project_span
import json
from shapely.geometry import Point,shape
import os
from datetime import datetime,timedelta
from functions import Rmax_calc,recovery_calc,month_index
import scipy.optimize
from geojson import Feature, FeatureCollection, Polygon

# Read files

In [2]:
# blocks information
file_path = os.path.join('..','..','Resources','blocks_info.csv')
blocks_df = pd.read_csv(file_path)
# actual gold produciton by month
file_path = os.path.join('..','..','Resources','act_monthly_oz.csv')
act_oz_df = pd.read_csv(file_path)

# Recovery Calculation

In [3]:
# Convert to datetime format
blocks_df['stack_finish'] = pd.to_datetime(blocks_df['stack_finish'])
blocks_df['leach_start'] = pd.to_datetime(blocks_df['leach_start'])
blocks_df['leach_end'] = pd.to_datetime(blocks_df['leach_end'])
blocks_df['leach_days'] = blocks_df['leach_end']-blocks_df['leach_start']
lat_lng_list = blocks_df[['lat','lng']].values.tolist()
# Create a dataframe with sequence as block_id, lift and lat_lng as columns
block_leach_time = pd.DataFrame(blocks_df[['lift']])
block_leach_time['lat_lng']=blocks_df[['lat','lng']].values.tolist()
block_leach_time.index.name = 'block_id'
# Create block_list for later index reference.
block_list = block_leach_time.index.tolist()

In [4]:
# Set up project span
projectspan = (
    datetime.strptime(project_span['startdate'],'%Y-%m-%d'),
    datetime.strptime(project_span['enddate'],'%Y-%m-%d')
    )
# Create month list of the project
startyear = projectspan[0].year
startmonth = projectspan[0].month
endyear = projectspan[1].year
endmonth = projectspan[1].month
monthlist = [datetime(m//12, m%12+1, 1) for m in range(startyear*12+startmonth-1, endyear*12+endmonth)]
# Use monthlist create recovery dataframe
rec_df = pd.DataFrame(monthlist,columns = ['months'])

### Simplify code by creating dataframes first and then apply functions to dataframes (if have time)

In [5]:
# This block of code calculates a specific block's recovery over months.
## The function takes two block indices 
## and the k factor that is going to be used for recovery calculation.
def block_months_rec(i,ref,k,lift_diff):
    block = blocks_df.iloc[i]
    # if the two indices are the same, means this is calculating the primary leach recovery
    if i == ref:
        start_date = block['leach_start']
        end_date = block['leach_end']
    # if the two indices are different, means calculating recoveries of blocks under secondary leach
    else:
        # ref_block is the block on the very top
        ref_block = blocks_df.iloc[ref]
        # the blocks below the ref_block inherit the leaching dates from the ref_block
        # calculate leach delay
        start_date = ref_block['leach_start'] + timedelta(days = t_sat + round(lift_diff * flow_rate))
        end_date = ref_block['leach_end'] + timedelta(days = round(lift_diff * flow_rate))
    # locate the month row
    m_index = month_index(start_date,projectspan)
    start_month = start_date.month
    end_month = end_date.month + (end_date.year - start_date.year) * 12
    rmax = block['rmax']
    # calcuate how much recovery that this block has achieved since the start
    rcum = sum(rec_df.iloc[0:m_index][i])
    if rcum < rmax: # if cumulative recovery has not achieved the maximum recovery
        for m in range(start_month,end_month+1):
            # calculate how many days in this month for recovery calculation
            if m == start_month:
                if m == 12:
                    days = 31 - start_date.day
                else:
                    days = (datetime(start_date.year,start_date.month+1,1)-start_date).days
            elif m == end_month:
                days = end_date.day
            elif m < 12:
                days = (datetime(start_date.year,m+1,1)-datetime(start_date.year,m,1)).days
            elif m == 12:
                days = 31
            else:
                days = (datetime(start_date.year+1,m-12+1,1)-datetime(start_date.year+1,m-12,1)).days
            # recovery calculation here.
            r = recovery_calc(rmax,rcum,k,days)
            if rcum + r > rmax:
                r = rmax - rcum
                rcum = 1
            else:
                rcum += r
            # write the recovery to the rec_df.
            rec_df.at[m_index,i] = r
            m_index += 1

In [6]:
records = blocks_df.to_records(index=False)

In [7]:
records[0]['lat']

65.012

In [8]:
## k factor here will be changed to a formula
k = 0.001
def blocks_recoveries(k):
    for high in block_list:
        this_block = blocks_df.iloc[high]
        rec_df[high]=0.0
        lift_diff = 0
        block_months_rec(high,high,k,lift_diff)
        high_lift = this_block['lift']
        # After this block's recovery has been calculated,
        # start calculating any blocks that are below this block.
        for low in range(0,high):
            if lat_lng_list[high]==lat_lng_list[low]:
                low_lift = blocks_df.iloc[low]['lift']
                lift_diff = high_lift-low_lift
                block_months_rec(low,high,k,lift_diff)

blocks_recoveries(k)

In [9]:
# Create dataframe records ounces leached out from each block during months
def rec_to_ozs():
    oz_df = pd.DataFrame(rec_df.loc[0:,0:] * blocks_df.ounces_per_block)
    oz_df.insert(0, 'months', monthlist)
    oz_df['oz_monthly'] = oz_df.sum(axis = 1)
    return oz_df
oz_df = rec_to_ozs()

In [10]:
def monthly_ozs(k):
    blocks_recoveries(k)
    rec_to_ozs()
    return oz_df['oz_monthly']

In [11]:
# This function is to calculate the remaining ounces of blocks without leaching
def init_oz(X):
    # input X is the dataframe's row in a series
    m_place = month_index(X[-2],projectspan)
    Y = X.copy()
    Y[m_place + 2:-2] = Y[-1]
    return Y

In [12]:
# Creating cells gold production stats by month.
blocks_oz_df = oz_df.set_index('months').T
## Export the data for visualization
file_path = os.path.join('..','..','Resources','blocks_oz.json')
blocks_oz_df.to_json(file_path)
blocks_oz_df.drop(blocks_oz_df.tail(1).index,inplace=True)
## Insert cell column for groupby.
blocks_oz_df.insert(0,'cell',blocks_df['cell'])
cell_oz_df = blocks_oz_df.groupby(['cell']).sum()
## Export the data for visualization
file_path = os.path.join('..','..','Resources','cells_oz.json')
cell_oz_df.to_json(file_path)
# Create gold production by position
blocks_oz_df.insert(0,"lat",blocks_df['lat'])
blocks_oz_df.insert(1,"lng",blocks_df['lng'])
position_oz_df = blocks_oz_df
position_oz_df.drop(columns = 'cell',inplace = True)
position_oz_df = position_oz_df.groupby(['lat','lng']).sum()
position_oz_df.reset_index(inplace = True)
# Blocks cumulative gold production
cumsumlist = blocks_oz_df.columns.tolist()[2:]
blocks_cumsum_oz_df = blocks_oz_df.copy()
blocks_cumsum_oz_df[cumsumlist] = blocks_cumsum_oz_df[cumsumlist].cumsum(axis = 1)
blocks_cumsum_oz_df.head()

months,lat,lng,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
0,65.012000,-147.351429,0.0,0.0,0.016734,0.262195,0.505379,0.731209,0.934031,0.934031,...,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031
1,65.012000,-147.350612,0.0,0.0,0.016734,0.262195,0.505379,0.731209,0.934031,0.934031,...,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031
2,65.012265,-147.353061,0.0,0.0,0.016734,0.262195,0.505379,0.731209,0.934031,0.934031,...,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031
3,65.012265,-147.352245,0.0,0.0,0.016734,0.262195,0.505379,0.731209,0.934031,0.934031,...,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031
4,65.012265,-147.351429,0.0,0.0,0.016734,0.262195,0.505379,0.731209,0.934031,0.934031,...,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031,0.934031


In [13]:
# Blocks remaining ounces
blocks_remaining_oz_df = blocks_df[['lat','lng']]
blocks_remaining_oz_df[cumsumlist] = 0.0
blocks_remaining_oz_df[['stack_finish','ounces_per_block']] = blocks_df[['stack_finish','ounces_per_block']]
blocks_remaining_oz_df = blocks_remaining_oz_df.apply(init_oz,axis=1)
blocks_remaining_oz_df.drop(columns = ['stack_finish','ounces_per_block'],inplace = True)
blocks_remaining_oz_df[cumsumlist]=blocks_remaining_oz_df[cumsumlist]-blocks_cumsum_oz_df[cumsumlist]

# Position remaining ounces
position_remaining_df = blocks_remaining_oz_df.groupby(['lat','lng']).sum()
position_remaining_df.reset_index(inplace = True)
position_remaining_df.tail()


<ipython-input-13-0d188f430265>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blocks_remaining_oz_df[cumsumlist] = 0.0
C:\Users\dzhang\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\dzhang\Anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,lat,lng,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
1429,65.024204,-147.369388,0.0,0.0,0.0,0.0,0.0,0.000000,4.992853,4.911867,...,4.704892,4.704892,4.704892,4.704892,4.704892,4.704892,4.704892,4.704892,4.704892,4.704892
1430,65.024204,-147.368571,0.0,0.0,0.0,0.0,0.0,11.772689,11.592210,11.430277,...,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277
1431,65.024204,-147.367755,0.0,0.0,0.0,0.0,0.0,11.772689,11.592210,11.430277,...,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277
1432,65.024204,-147.366939,0.0,0.0,0.0,0.0,0.0,11.772689,11.592210,11.430277,...,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277
1433,65.024204,-147.366122,0.0,0.0,0.0,0.0,0.0,11.772689,11.592210,11.430277,...,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277,11.430277


In [14]:
# Convert datetime columns to string
date_list = position_remaining_df.columns.tolist()[2:]
converted_date_list = [str(date.year)+'-'+str(date.month) if date.month>=10 else str(date.year)+'-0'+str(date.month) for date in date_list]
convert_dict = dict(zip(date_list, converted_date_list))
position_remaining_df.rename(columns = convert_dict,inplace = True)

In [26]:
# Geojson
# Define block size
delta_lat = (lat_max - lat_min)/resolution[0]/2
delta_lng = (lng_max - lng_min)/resolution[1]/2
features = position_remaining_df.apply(
    lambda row: Feature(geometry=Polygon([[
        # (row['lat'] + delta_lat, row['lng'] + delta_lng),
        # (row['lat'] - delta_lat, row['lng'] + delta_lng),
        # (row['lat'] - delta_lat, row['lng'] - delta_lng),
        # (row['lat'] + delta_lat, row['lng'] - delta_lng),
        # (row['lat'] + delta_lat, row['lng'] + delta_lng)
        (row['lng'] + delta_lng, row['lat'] + delta_lat),
        (row['lng'] - delta_lng, row['lat'] + delta_lat),
        (row['lng'] - delta_lng, row['lat'] - delta_lat),
        (row['lng'] + delta_lng, row['lat'] - delta_lat),
        (row['lng'] + delta_lng, row['lat'] + delta_lat)
    ]])),
    axis=1).tolist()

properties = position_remaining_df.drop(['lat', 'lng'], axis=1).to_dict('records')

for x in range(len(features)):
    features[x].properties = properties[x]

feature_collection = FeatureCollection(features=features)

file_path = os.path.join('..','..','Resources','position_remain_oz.geojson')

with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

In [16]:
position_oz_df.tail()

months,lat,lng,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
1429,65.024204,-147.369388,0.0,0.0,0.0,0.0,0.0,0.000000,0.010718,0.080986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1430,65.024204,-147.368571,0.0,0.0,0.0,0.0,0.0,0.017888,0.180479,0.161933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1431,65.024204,-147.367755,0.0,0.0,0.0,0.0,0.0,0.017888,0.180479,0.161933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1432,65.024204,-147.366939,0.0,0.0,0.0,0.0,0.0,0.017888,0.180479,0.161933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1433,65.024204,-147.366122,0.0,0.0,0.0,0.0,0.0,0.017888,0.180479,0.161933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
position_oz_df['overall_ozs'] = position_oz_df.drop(columns = ['lat','lng']).sum(axis = 1)
position_oz_df.head()

months,lat,lng,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,...,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00,overall_ozs
0,65.012000,-147.351429,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934031
1,65.012000,-147.350612,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934031
2,65.012265,-147.353061,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934031
3,65.012265,-147.352245,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934031
4,65.012265,-147.351429,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934031


In [18]:
file_path = os.path.join('..','..','Resources','latlng_oz.json')
position_oz_df.to_json(file_path)

In [50]:
# # Create monthly production file
# monthly_production = blocks_oz_df.sum()
# file_path = os.path.join('..','..','Resources','monthly_oz.json')
# monthly_production.to_json(file_path)
# file_path = os.path.join('..','..','Resources','monthly_oz.csv')
# monthly_production.to_csv(file_path)

In [51]:
# create dataframe for each blocks' remaining ounces over months.
#remain_oz_df

In [52]:
# Creating blocks cumulative gold production
#cum_oz_df=oz_df[]
#cum_oz_df.head()

In [30]:
#production = f(k,dataframe)
#f(production, actual_production) = k

# Optimize K

In [82]:
see_this = monthly_ozs(0.001)
np.nansum((act_oz_df['ounces']-see_this)**2)

38529.88818027917

In [98]:
def sq(k):
    # use this k to calculate gold production
    model_rec = monthly_ozs(k)
    actual_rec = act_oz_df['ounces']
    delta = np.nansum((actual_rec-model_rec)**2)
    # delta = abs(calculated recovery - actual recovery)
    return delta
# Check other options and see steps.
x = scipy.optimize.minimize(sq,0.2)
print(x)

C:\Users\dzhang\Anaconda3\lib\site-packages\scipy\optimize\nonlin.py:1016: RuntimeWarning: divide by zero encountered in true_divide
  d = v / vdot(df, v)


KeyboardInterrupt: 

In [ ]:
new_delta = sq(x)
print(new_delta)